<a href="https://colab.research.google.com/github/SomdeepAcharyya/NLP/blob/main/LDA_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import io
import pandas as pd
import spacy
from textblob import TextBlob
from spacy.lang.en import English
nlp = English()
import csv
from itertools import chain
from spacy.lang.en.stop_words import STOP_WORDS
import nltk
nltk.download('wordnet')
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
spc = spacy.load('en_core_web_sm')
import http.client
import urllib.parse
from urllib.request import urlopen
import math
!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import datetime
from time import strptime

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# read csv 
import pandas as pd
path = r'/content/drive/MyDrive/Colab Notebooks/Rumour/datasets/germanwings_crash_tweets_text.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  ts = pd.read_csv(infile)
print(ts.shape)

(4489, 84)


In [ ]:
ts.text

0       French interior ministry: Debris from #GermanW...
1                                                        
2                                                        
3       @ElisabethRMeier @SkyNews yeah i have, thanks ...
4                                                        
                              ...                        
4484                        @airlivenet Never trust @BILD
4485                                                     
4486    @airlivenet If you want to be reliable never r...
4487    @airlivenet Correction: "@BILD" is not a newsp...
4488                                                     
Name: text, Length: 4489, dtype: object

In [ ]:
# Load the regular expression library
import re
papers = pd.DataFrame()
papers['paper_text'] = ts['text']
# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

0    french interior ministry: debris from #germanw...
1                                                     
2                                                     
3    @elisabethrmeier @skynews yeah i have thanks f...
4                                                     
Name: paper_text_processed, dtype: object

In [ ]:
x = papers[papers['paper_text_processed'] != ' ']
x

,paper_text,paper_text_processed
0,French interior ministry: Debris from #GermanW...,french interior ministry: debris from #germanw...
3,"@ElisabethRMeier @SkyNews yeah i have, thanks ...",@elisabethrmeier @skynews yeah i have thanks f...
5,@SkyNews Oh God. 😱,@skynews oh god 😱
6,@SkyNews man this sucks. And I'm flying to Bar...,@skynews man this sucks and i'm flying to barc...
7,What we know about low-cost airline Germanwing...,what we know about low-cost airline germanwing...
...,...,...
4482,NEWS #Germanwings co-pilot Andreas Lubitz had ...,news #germanwings co-pilot andreas lubitz had ...
4483,@airlivenet Bild newspaper is yellow press and...,@airlivenet bild newspaper is yellow press and...
4484,@airlivenet Never trust @BILD,@airlivenet never trust @bild
4486,@airlivenet If you want to be reliable never r...,@airlivenet if you want to be reliable never r...


In [ ]:
lemmatizer = WordNetLemmatizer()
punctuation = re.compile(r'[-.?&!,:;()|0-9]\/`'"'")
def get_lemma_words(t):
    #t = t.lower()
    #a = nlp(t)
    lemmat= " "
    stop_words = set(stopwords.words('english'))  
    word_tokens = word_tokenize(p.clean(t)) 
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    punc_words = []
    for words in filtered_sentence:
        word =  punctuation.sub("", words)
        if len(word) > 0:
          punc_words.append(word)
    for word in punc_words:
        lemmat = lemmat + " " + (lemmatizer.lemmatize(word))
    #print('y')
    return lemmat

In [ ]:
x['lemma'] = ' '
for i in range(len(x)):
  y = get_lemma_words(re.sub('\W+',' ',x['paper_text_processed'][i]))
  x['lemma'][i] = y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
x

,level_0,index,paper_text,paper_text_processed,lemma
0,0,0,French interior ministry: Debris from #GermanW...,french interior ministry: debris from #germanw...,french interior ministry debris germanwings ...
1,1,3,"@ElisabethRMeier @SkyNews yeah i have, thanks ...",@elisabethrmeier @skynews yeah i have thanks f...,elisabethrmeier skynews yeah thanks asking
2,2,5,@SkyNews Oh God. 😱,@skynews oh god 😱,skynews oh god
3,3,6,@SkyNews man this sucks. And I'm flying to Bar...,@skynews man this sucks and i'm flying to barc...,skynews man suck flying barcelona 2 week
4,4,7,What we know about low-cost airline Germanwing...,what we know about low-cost airline germanwing...,know low cost airline germanwings whose plan...
...,...,...,...,...,...
3033,3033,4482,NEWS #Germanwings co-pilot Andreas Lubitz had ...,news #germanwings co-pilot andreas lubitz had ...,news germanwings co pilot andreas lubitz ser...
3034,3034,4483,@airlivenet Bild newspaper is yellow press and...,@airlivenet bild newspaper is yellow press and...,airlivenet bild newspaper yellow press sourc...
3035,3035,4484,@airlivenet Never trust @BILD,@airlivenet never trust @bild,airlivenet never trust bild
3036,3036,4486,@airlivenet If you want to be reliable never r...,@airlivenet if you want to be reliable never r...,airlivenet want reliable never requote bild ...


In [ ]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = x.lemma.values.tolist()
data_words = list(sent_to_words(data))
print(data_words)

[['french', 'interior', 'ministry', 'debris', 'germanwings', 'airbus', 'altitude', 'http', 'tco', 'upmsinqkx', 'http', 'tco', 'miu', 'nhnbr'], ['elisabethrmeier', 'skynews', 'yeah', 'thanks', 'asking'], ['skynews', 'oh', 'god'], ['skynews', 'man', 'suck', 'flying', 'barcelona', 'week'], ['know', 'low', 'cost', 'airline', 'germanwings', 'whose', 'plane', 'crashed', 'french', 'alp', 'today', 'http', 'tco', 'dpvjegkegt', 'http', 'tco', 'hkxdditls'], ['mashable', 'fl', 'germanwings', 'week', 'ago', 'done', 'often', 'connecting', 'lh', 'flight', 'frankfurt', 'good', 'crew', 'equipment'], ['mashable', 'defect', 'security', 'pilot', 'error'], ['mashable'], ['buen', 'resumen', 'lo', 'que', 'hay', 'que', 'saber', 'de', 'la', 'aerolnea', 'bajo', 'coste', 'germanwing', 'via', 'mashable', 'http', 'tco', 'cqn', 'rfcyw'], ['mashable', 'way', 'create', 'bias', 'calling', 'low', 'cost', 'mean', 'apt', 'crash'], ['germanwings', 'airbus', 'way', 'barcelona', 'dusseldorf', 'disappeared', 'radar', 'am', '

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])

[[]]


In [ ]:
data_lemmatized

[[],
 ['skynew', 'thank', 'ask'],
 [],
 ['fly'],
 ['know',
  'low',
  'cost',
  'airline',
  'germanwing',
  'plane',
  'crash',
  'alp',
  'today'],
 ['mashable',
  'germanwing',
  'week',
  'ago',
  'do',
  'often',
  'connect',
  'flight',
  'good',
  'crew',
  'equipment'],
 ['mashable', 'defect', 'security', 'pilot', 'error'],
 ['mashable'],
 ['germanwe', 'mashable'],
 ['mashable', 'way', 'create', 'bias', 'call', 'low', 'cost', 'crash'],
 ['disappear', 'radar', 'local', 'time'],
 ['cbsthismorne', 'disappear'],
 ['update', 'yesterday', 'take', 'exactly', 'route', 'planefinder', 'http'],
 ['plane',
  'typically',
  'fly',
  'route',
  'flight',
  'plan',
  'deviation',
  'weather'],
 ['plane',
  'already',
  'do',
  'day',
  'previous',
  'wonder',
  'take',
  'route',
  'first',
  'place'],
 ['look',
  'control',
  'forward',
  'pick',
  'speed',
  'desperate',
  'pull',
  'control',
  'back',
  'get',
  'low'],
 ['everywhere'],
 ['germanwing', 'old', 'serial', 'number', 'deliver'

In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus)

[[], [(0, 1), (1, 1), (2, 1)], [], [(3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1)], [(20, 1), (23, 1), (24, 1), (25, 1), (26, 1)], [(20, 1)], [(20, 1), (27, 1)], [(6, 1), (7, 1), (10, 1), (20, 1), (28, 1), (29, 1), (30, 1), (31, 1)], [(32, 1), (33, 1), (34, 1), (35, 1)], [(32, 1), (36, 1)], [(37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)], [(3, 1), (11, 1), (18, 1), (40, 1), (44, 1), (45, 1), (46, 1), (47, 1)], [(11, 1), (16, 1), (40, 1), (41, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1)], [(10, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1)], [(63, 1)], [(8, 1), (64, 1), (65, 1), (66, 1), (67, 1)], [(8, 1), (64, 1), (68, 1)], [(64, 1), (68, 1)], [(35, 1)], [(69, 1)], [(38, 1), (70, 1), (71, 1), (72, 2), (73, 1), (74, 1)], [(7, 1), (75, 2), (76, 1), (77, 1), (78, 1)], [], [(64, 1), (65, 

In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
from pprint import pprint
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.026*"fast" + 0.025*"check" + 0.022*"clean" + 0.021*"lindasharkey" + 0.020*"site" + 0.016*"expect" + 0.015*"fear" + 0.014*"due" + 0.014*"glide" + 0.014*"best"'), (1, '0.148*"say" + 0.073*"bbcbreake" + 0.045*"french" + 0.026*"mashable" + 0.025*"soon" + 0.019*"read" + 0.016*"minute" + 0.016*"way" + 0.016*"least" + 0.016*"people"'), (2, '0.046*"never" + 0.042*"go" + 0.039*"see" + 0.034*"strong" + 0.023*"seem" + 0.022*"tell" + 0.018*"be" + 0.015*"happen" + 0.014*"really" + 0.013*"time"'), (3, '0.054*"people" + 0.046*"send" + 0.033*"get" + 0.027*"could" + 0.026*"kill" + 0.024*"terrorist" + 0.024*"bad" + 0.023*"come" + 0.020*"survivor" + 0.018*"apparently"'), (4, '0.190*"crash" + 0.145*"plane" + 0.111*"germanwing" + 0.058*"break" + 0.053*"http" + 0.053*"flight" + 0.025*"german" + 0.017*"confirm" + 0.016*"tco" + 0.015*"mayday"'), (5, '0.084*"passenger" + 0.046*"want" + 0.045*"think" + 0.035*"crew" + 0.035*"may" + 0.033*"would" + 0.018*"take" + 0.017*"wing" + 0.015*"terror" + 0.015*"ter

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.566768140606127


In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  0%|          | 0/540 [00:00<?, ?it/s]

ValueError: ignored

In [ ]:
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
!pip install pyLDAvis
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared